In [ ]:
import csv
import pandas as pd
import numpy as np
import random

from google.colab import files

uploaded = files.upload()

Saving A_Christmas_Carol.xlsx to A_Christmas_Carol.xlsx
Saving Collision_of_Worlds_1.xlsx to Collision_of_Worlds_1.xlsx
Saving Collision_of_Worlds_2.xlsx to Collision_of_Worlds_2.xlsx
Saving Collision_of_Worlds_3.xlsx to Collision_of_Worlds_3.xlsx
Saving Collision_of_Worlds_4.xlsx to Collision_of_Worlds_4.xlsx
Saving Collision_of_Worlds_5.xlsx to Collision_of_Worlds_5.xlsx
Saving Falling_Sentences.xlsx to Falling_Sentences.xlsx
Saving Frankenstein.xlsx to Frankenstein.xlsx
Saving In_Amundsen’s_Tent.xlsx to In_Amundsen’s_Tent.xlsx
Saving Leiningen_Versus_the_Ants.xlsx to Leiningen_Versus_the_Ants.xlsx
Saving Observer_1-A_Warm_Home.xlsx to Observer_1-A_Warm_Home.xlsx
Saving Observer_2-Charity.xlsx to Observer_2-Charity.xlsx
Saving Observer_3-One_of_Us.xlsx to Observer_3-One_of_Us.xlsx
Saving Observer_4-Legends.xlsx to Observer_4-Legends.xlsx
Saving Prince_and_the_Pauper.xlsx to Prince_and_the_Pauper.xlsx
Saving The_Demon_King.xlsx to The_Demon_King.xlsx
Saving The_Exiled_Queen.xlsx to Th

In [ ]:
stories = {}
dataset = []

MEAN = 2  # Mean number of sentences radius
STD_DV = 2  # Standard deviation size
MIN_RADIUS = 1  # Minimum radius size
MAX_RADIUS = 6  # Maximum radius size

TOKEN_NEIGHBORHOOD_RADIUS = 30 # These aren't implemnented fully yet
TOKEN_LENGTH = 4

for file_name, file_data in uploaded.items():
    try:
        df = pd.read_excel(file_data, header=None)

        # Check for annotation
        if 1 not in df[0].values:
          print(f"No scene transition found in {file_name}, skipping...")
          continue

        # Find the index of the first '1' in the first column
        first_scene_transition_index = df[df[0] == 1].index[0]

        # Replace newlines in the sentences column
        df[1] = df[1].astype(str).apply(lambda x: x.replace('\n', ' '))

        # fill any missed entries with 0
        df[0] = df[0].fillna(0)

         # Remove rows where the sentence is empty
        df = df.dropna(subset=[1])
        df = df[df[1] != 0]
        df = df[df[1] != '0']

        # # Slice the DataFrame to keep only the rows from the first '1' onwards
        df = df.iloc[first_scene_transition_index:].reset_index(drop=True)

        stories[file_name] = df
    except Exception as e:
        print(f"Error reading {file_name}: {e}")



No scene transition found in Prince_and_the_Pauper.xlsx, skipping...
No scene transition found in The_Legend_of_Sleepy_Hollow.xlsx, skipping...
No scene transition found in There_Will_Come_Soft_Rains.xlsx, skipping...


In [ ]:
def print_neighborhood_info(neighborhood, start_idx, end_idx):

    print(f"Neighborhood from index {start_idx} to {end_idx} (Length: {end_idx - start_idx} sentences):")
    for index, row in neighborhood.iterrows():
        print(f"Index: {index}, Label: {row[0]} : {row[1]} ")
    print("------\n")

def generate_gaussian_window_size(mean, std_dev, min_size, max_size):
    size = round(np.random.normal(mean, std_dev))
    return max(min_size, min(size, max_size))

def flatten_control(neighborhood):
    filtered_sentences = neighborhood[1].dropna().tolist()

    # Join the sentences into a single string with spaces
    concatenated_sentences = ' '.join(filtered_sentences)

    # Create and return DataFrame with a single entry
    return pd.DataFrame({'isTransition': [0], 'text': [concatenated_sentences]})

In [ ]:
def generate_sentence_neighborhoods(df, mean_radius, std_dev_radius, min_radius, max_radius):

    """
    Generates sentence neighborhoods centered around transition points in a DataFrame.

    Parameters:
    - df (pandas.DataFrame) : Assumption is that the first row is a scene transition
    - radius : Number of sentences around the transition to include (before offsets)

    Returns:
    - list of pandas.DataFrame: A list containing the neighborhoods as DataFrames with one entry. A flattening of the neighborhood

    Notes:
    - For adjacent transitions (or story beginnings) only one neighborhood in the direction opposite to the conflict is returned
    - Only neighborhoods that contain at least one sentence before or after the transition (not including
      the transition itself) are included in the output.
    """

    neighborhoods = []

    transition_indices = df.index[df[0] == 1].tolist()  # Scene transition indices

    for idx in transition_indices:
      radius = generate_gaussian_window_size(mean_radius, std_dev_radius, min_radius, max_radius)
      length = ( radius * 2 ) + 1

      # These are the number of rows I can crawl up and down without hitting another transition
      max_prev_distance = get_distance_to_prev_transition(df, idx) - 1
      max_next_distance = get_distance_to_next_transition(df, idx) - 1

      # Indices of next and previous transitions
      prev_transition_index = idx - get_distance_to_prev_transition(df, idx)
      next_transition_index = idx + get_distance_to_next_transition(df, idx)

      # Define the start and end indices of the neighborhood
      start_idx = idx - min(radius, max_prev_distance, max_next_distance)
      end_idx = idx + min(radius, max_prev_distance, max_next_distance)

      # Offset is where we define where we place the transition in the quartile, currently it is centered, rounding down
      # We could add a little randomness here
      neighborhood_size = end_idx - start_idx
      offset = neighborhood_size // 3

      # In cases with adjacent transitions this bonus allows us to grab at least one neighborhood where our transition is on the edge and we go out one radius length away from the other transition
      if (max_next_distance == 0):
        backward_bonus = radius
      else :
        backward_bonus = 0
      if (max_prev_distance == 0):
        forward_bonus = radius
      else :
        forward_bonus = 0
      if (max_next_distance == 0 and max_prev_distance == 0):
        continue

      # Truncate this neighborhood if our desired offset captures another transition
      if (prev_transition_index >= (start_idx - offset)):
        last_third_start = prev_transition_index + 1
      else :
        last_third_start = start_idx - offset

      if (next_transition_index <= (end_idx + offset + 1)):
        first_third_end = next_transition_index - 1
      else :
        first_third_end = end_idx + offset

      #It's not applying the backward bonus correctly

      # Capture neighborhoods
      first_third = df.iloc[start_idx + offset : first_third_end + 1 + forward_bonus]
      mid_third = df.iloc[start_idx : end_idx + 1]
      last_third = df.iloc[last_third_start - backward_bonus: end_idx - offset + 1]

      # Make sure the transition isn't alone, then flatten
      if (first_third[0] == 0).any():
        # print_neighborhood_info(first_third, start_idx + offset, first_third_end + 1 + forward_bonus)
        neighborhoods.append(flatten(first_third))

      if (mid_third[0] == 0).any():
        if not mid_third.equals(first_third):
          # print_neighborhood_info(mid_third, start_idx, end_idx + 1)
          neighborhoods.append(flatten(mid_third))

      if (last_third[0] == 0).any():
        if not last_third.equals(first_third) and not last_third.equals(mid_third):
          # print_neighborhood_info(last_third, last_third_start - backward_bonus, end_idx - offset + 1)
          neighborhoods.append(flatten(last_third))

    return neighborhoods


def get_distance_to_prev_transition(df, index) :
  if index == 0:      # Start of story
        return 1
  for i in range(index - 1, -1, -1):
      if df.iloc[i, 0] == 1:
          return index - i
  return None

def get_distance_to_next_transition(df, index) :
    for i in range(index + 1, len(df)):
        if df.iloc[i, 0] == 1:
            return i - index
    return len(df) - index # Count the end of the story as the next scene transition

def pad_neighborhood(neighborhood, length):
    rows_to_add = length - len(neighborhood)

    # If the DataFrame needs padding
    if rows_to_add > 0:
        padding = pd.DataFrame(np.nan, index=range(rows_to_add), columns=neighborhood.columns)
        neighborhood = pd.concat([neighborhood, padding], ignore_index=True)

    return neighborhood

def flatten(neighborhood):
    filtered_sentences = neighborhood[1].dropna().tolist()

    # Join the sentences into a single string with spaces
    concatenated_sentences = ' '.join(filtered_sentences)

    # Create and return DataFrame with a single entry
    return pd.DataFrame({'isTransition': [1], 'text': [concatenated_sentences]})


In [ ]:
def generate_control_sentence_neighborhoods(df, mean_radius, std_dev_radius, min_radius, max_radius):

    non_transition_neighborhoods = []
    transition_indices = set(df.index[df[0] == 1].tolist())
    all_indices = set(range(len(df)))

    # Non-transition indices are all indices minus the transition indices
    non_transition_indices = list(all_indices - transition_indices)

    for idx in non_transition_indices:
        radius = generate_gaussian_window_size(mean_radius, std_dev_radius, min_radius, max_radius)
        start_idx = max(idx - radius, 0)
        end_idx = min(idx + radius, len(df) - 1)

        if not set(range(start_idx, end_idx + 1)).intersection(transition_indices):
            neighborhood = df.loc[start_idx:end_idx + 1].copy()

            # This corrects for cases when the annotator left two sentences separated by one newline, by truncating the second
            neighborhood.loc[:, 1] = neighborhood.loc[:, 1].apply(lambda x: x.split('\n')[0])

            if (neighborhood[0] == 0).all():  # Ensure no transitions within neighborhood
                # print_neighborhood_info(neighborhood, start_idx, end_idx)
                non_transition_neighborhoods.append(flatten_control(neighborhood.drop(columns=[0])))

    return non_transition_neighborhoods


In [ ]:
all_neighborhoods = []

for file_name, df in stories.items():
    print(f"Processing {file_name}...")
    try:
        neighborhoods = generate_sentence_neighborhoods(df, MEAN, STD_DV, MIN_RADIUS, MAX_RADIUS)
        all_neighborhoods.extend(neighborhoods)
        print(f"Added transition neighborhoods")

        control_neighborhoods = generate_control_sentence_neighborhoods(df, MEAN, STD_DV, MIN_RADIUS, MAX_RADIUS)

      # If control_neighborhoods is larger, drop random entries but maintain order
        while len(control_neighborhoods) > len(neighborhoods):
            # Randomly select an index to remove
            index_to_remove = random.randint(0, len(control_neighborhoods) - 1)
            # Remove the entry at the selected index
            del control_neighborhoods[index_to_remove]

        all_neighborhoods.extend(control_neighborhoods)
        print(f"Added control neighborhoods")
    except Exception as e:
        print(f"Error processing {file_name}: {e}")

# Concatenate all neighborhood DataFrames into a single DataFrame
if all_neighborhoods:
    neighborhoods_df = pd.concat(all_neighborhoods, ignore_index=True)
    neighborhoods_df.to_csv('mean=2_std=2_dataset.csv', index=False)
    print("All neighborhoods have been exported to 'mean=2_std=2_dataset'.")
    print(neighborhoods_df)
else:
    print("No neighborhoods to export.")

Processing A_Christmas_Carol.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Collision_of_Worlds_1.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Collision_of_Worlds_2.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Collision_of_Worlds_3.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Collision_of_Worlds_4.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Collision_of_Worlds_5.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Falling_Sentences.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Frankenstein.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing In_Amundsen’s_Tent.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Leiningen_Versus_the_Ants.xlsx...
Added transition neighborhoods
Added control neighborhoods
Processing Obser

In [ ]:

def generate_token_neighborhoods(df, radius=TOKEN_NEIGHBORHOOD_RADIUS, token_length=TOKEN_LENGTH):
  # My idea here is to grab the entire scene transition sentence, then instead of grabbing sentences around it, grab 4 token chunks backward and forward in the dataframe until we hit a partial word -- or maybe just keep the partial words

    neighborhoods = []
    length = ( radius * 2 ) + 1

    transition_indices = df.index[df[0] == 1].tolist()  # Scene transition indices

    for idx in transition_indices:

      # These are the number of rows I can crawl up and down without hitting another transition
      max_prev_distance = get_distance_to_prev_transition(df, idx) - 1
      max_next_distance = get_distance_to_next_transition(df, idx) - 1

      # Indices of next and previous transitions
      prev_transition_index = idx - get_distance_to_prev_transition(df, idx)
      next_transition_index = idx + get_distance_to_next_transition(df, idx)

      # Define the start and end indices of the neighborhood
      start_idx = idx - min(radius, max_prev_distance, max_next_distance)
      end_idx = idx + min(radius, max_prev_distance, max_next_distance)

      # Offset is where we define where we place the transition in the quartile, currently it is centered, rounding down
      # We could add a little randomness here
      neighborhood_size = end_idx - start_idx
      offset = neighborhood_size // 3

      # In cases with adjacent transitions this bonus allows us to grab at least one neighborhood where our transition is on the edge and we go out one radius length away from the other transition
      if (max_next_distance == 0):
        backward_bonus = radius
      else :
        backward_bonus = 0
      if (max_prev_distance == 0):
        forward_bonus = radius
      else :
        forward_bonus = 0
      if (max_next_distance == 0 and max_prev_distance == 0):
        continue

      # Truncate this neighborhood if our desired offset captures another transition
      if (prev_transition_index >= (start_idx - offset)):
        last_third_start = prev_transition_index + 1
      else :
        last_third_start = start_idx - offset

      if (next_transition_index <= (end_idx + offset + 1)):
        first_third_end = next_transition_index - 1
      else :
        first_third_end = end_idx + offset

      # Capture neighborhoods
      first_third = df.iloc[start_idx + offset : first_third_end + 1 + forward_bonus]
      mid_third = df.iloc[start_idx : end_idx + 1]
      last_third = df.iloc[last_third_start - backward_bonus: end_idx - offset + 1]

      # Make sure the transition isn't alone, then flatten
      if (first_third[0] == 0).any():
        neighborhoods.append(flatten(pad_neighborhood(first_third, length)))

      if (mid_third[0] == 0).any():
        neighborhoods.append(flatten(pad_neighborhood(mid_third, length)))

      if (last_third[0] == 0).any():
        neighborhoods.append(flatten(pad_neighborhood(last_third, length)))

    return neighborhoods


def get_distance_to_prev_transition(df, index) :
  if index == 0:      # Start of story
        return 1
  for i in range(index - 1, -1, -1):
      if df.iloc[i, 0] == 1:
          return index - i
  return None

def get_distance_to_next_transition(df, index) :
    for i in range(index + 1, len(df)):
        if df.iloc[i, 0] == 1:
            return i - index
    return len(df) - index # Count the end of the story as the next scene transition

def pad_neighborhood(neighborhood, length):
    rows_to_add = length - len(neighborhood)

    # If the DataFrame needs padding
    if rows_to_add > 0:
        padding = pd.DataFrame(np.nan, index=range(rows_to_add), columns=neighborhood.columns)
        neighborhood = pd.concat([neighborhood, padding], ignore_index=True)

    return neighborhood

def flatten(neighborhood):
    filtered_sentences = neighborhood[1].dropna().tolist()

    # Join the sentences into a single string with spaces
    concatenated_sentences = ' '.join(filtered_sentences)

    # Create and return DataFrame with a single entry
    return pd.DataFrame([concatenated_sentences])